This exercise consists in analyzing a dataset containg timing information from a series of Time-to-Digital-Converters (TDC) implemented in a pair of FPGAs. Each measurement (i.e. each row of the input file) consists of a flag that specifies the type of message ('HEAD', which in this case is always 1), two addresses of the TDC providing the signal ('FPGA' and 'TDC_CHANNEL'), and the timing information ('ORBIT_CNT', 'BX_COUNTER', and 'TDC_MEAS'). Each TDC count corresponds to 25/30 ns, whereas a unit of BX_COUNTER corresponds to 25 ns, and the ORBIT_CNT is increased every 'x' BX_COUNTER. This allows to store the time in a similar way to hours, minutes and seconds.

In [ ]:
# If haven't downloaded it yet, please get the data file with wget
#!wget https://www.dropbox.com/s/xvjzaxzz3ysphme/data_000637.txt -P ./data/

1\. Create a Pandas DataFrame reading N rows of the `data/data_000637.txt` dataset. Choose N to be smaller than or equal to the maximum number of rows and larger that 10k (check the documentation).

In [17]:
import pandas as pd
import numpy as np

#df = pd.read_csv('data_000637.txt', header = None)

#part 1
file= "./data_000637.txt"
#N = all rows
Data = np.genfromtxt(file, dtype='float64', delimiter=',', skip_header=1)

df = pd.DataFrame(Data, index=np.arange(len(Data)), columns=['HEAD','FPGA','TDC_CHANNEL','ORBIT_CNT','BX_COUNTER', 'TDC_MEAS'])
df






,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS
0,1.0,0.0,123.0,3.869200e+09,2374.0,26.0
1,1.0,0.0,124.0,3.869200e+09,2374.0,27.0
2,1.0,0.0,63.0,3.869200e+09,2553.0,28.0
3,1.0,0.0,64.0,3.869200e+09,2558.0,19.0
4,1.0,0.0,64.0,3.869200e+09,2760.0,25.0
...,...,...,...,...,...,...
1310715,1.0,0.0,62.0,3.869211e+09,762.0,14.0
1310716,1.0,1.0,4.0,3.869211e+09,763.0,11.0
1310717,1.0,0.0,64.0,3.869211e+09,764.0,0.0
1310718,1.0,0.0,139.0,3.869211e+09,769.0,0.0


2\. Estimate the number of BX in a ORBIT (the value 'x').

In [18]:
X = df["BX_COUNTER"].max()
print('X: ', X)

X:  3563.0


3\. Find out the duration of the data taking in hours, minutes and seconds. You can either make an estimate based on the fraction of the measurements (rows) you read, or perform this check precisely by reading the whole dataset.

In [19]:
df['Time_ns'] = df['TDC_MEAS']*25/30 + df['BX_COUNTER']*25 + (X*25)*df['ORBIT_CNT']
timens_min = np.min(df['Time_ns'])
timens_max = np.max(df['Time_ns'])
time_t = timens_max - timens_min
print('Time in n seconds:', time_t)

Time in n seconds: 980141153.3125


4\. Create a new column with the absolute time in ns (as a combination of the other three columns with timing information) since the beginning of the data acquisition.

In [20]:
df['Time_ns'] = df['TDC_MEAS']*25/30 + df['BX_COUNTER']*25 + (X*25)*df['ORBIT_CNT']
df

,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS,Time_ns
0,1.0,0.0,123.0,3.869200e+09,2374.0,26.0,3.446490e+14
1,1.0,0.0,124.0,3.869200e+09,2374.0,27.0,3.446490e+14
2,1.0,0.0,63.0,3.869200e+09,2553.0,28.0,3.446490e+14
3,1.0,0.0,64.0,3.869200e+09,2558.0,19.0,3.446490e+14
4,1.0,0.0,64.0,3.869200e+09,2760.0,25.0,3.446490e+14
...,...,...,...,...,...,...,...
1310715,1.0,0.0,62.0,3.869211e+09,762.0,14.0,3.446500e+14
1310716,1.0,1.0,4.0,3.869211e+09,763.0,11.0,3.446500e+14
1310717,1.0,0.0,64.0,3.869211e+09,764.0,0.0,3.446500e+14
1310718,1.0,0.0,139.0,3.869211e+09,769.0,0.0,3.446500e+14


5\. Use the `.groupby()` method to find out the noisy channels, i.e. the TDC channels with most counts (print to screen the top 3 and the corresponding counts)

In [21]:
df5 = (df.groupby('TDC_CHANNEL').sum()).sort_values(by="HEAD", ascending=False)
df5[['HEAD']].head(3)

,HEAD
TDC_CHANNEL,
139.0,108059.0
64.0,66020.0
63.0,64642.0


6\. Count the number of non-empty orbits (i.e. the number of orbits with at least one hit). Count also the number of unique orbits with at least one measurement from TDC_CHANNEL=139.

In [14]:
data = pd.read_csv('data_000637.txt')
non_empty_orbits = data['ORBIT_CNT'].unique() 
print('Non-empty orbits: ', non_empty_orbits.shape[0])

tdc_channel139 = data[data['TDC_CHANNEL'] == 139]
print('unique orbits from TDC_CHANNEL=139: ', tdc_channel139['ORBIT_CNT'].unique().shape[0])

Non-empty orbits:  11001
unique orbits from TDC_CHANNEL=139:  10976


7\. **Optional:** Make two occupancy plots (one for each FPGA), i.e. plot the number of counts per TDC channel